# Sentence Variational Adversarial Active Learning (S-VAAL)
@author: Tyler Bikaun

The following notebook intends to flesh out an initial proof concept of the coupling of S-VAE (Bowman <i>et al.</i> 2016; https://arxiv.org/abs/1511.06349) and VAAL (Sinha <i>et al.</i> 2019; https://arxiv.org/abs/1904.00370)

<b>Application:</b> Named Entity Task (NER)

<b>Architecture Diagram:</b>

### Standard Imports

In [1]:
import random
import numpy as np
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim

torch.manual_seed(1)

## Testing Functions
These functions are used for spot testing code whilst developing. For example, building random sequences of tensors.
- [x] Build artificial sequence generator

In [23]:
class Tester:
    def __init__(self):
        pass
    
    def build_sequences(self, batch_size: int, max_seq_len: int) -> int:
        """
        Builds tensor of specified size containing variable length, padded, sequences of integers
            
        Arguments
        ---------
            batch_size : int
                Number of sequences to generate
            max_seq_len : int
                Maximum length of sequences
        Returns
        -------
            sequences : tensor
                Tensor of generated sequences
            lengths : tensor
                Tensor of sequence lengths
        """
        seqs = list()
        for i in range(batch_size):
            # Generate random integer sequences
            seq = np.random.randint(low=1, high=100, size=(random.randint(1, max_seq_len),))
            # Add padding
            seq = np.concatenate((seq, np.zeros(shape=(max_seq_len - len(seq)))), axis=None)
            seqs.append(seq)
        sequences = torch.LongTensor(seqs)
        lengths = torch.tensor([len(seq[seq != 0]) for seq in sequences])
        
        print(f'Shapes - seq {sequences.shape} - lengths {lengths.shape}')
        
        return sequences, lengths
    
    def build_sequence_tags(self, sequences, label_space_size: int):
        """
        Given a set of sequences, generates ground truth labels
        
        Arguments
        ---------
            sequences : tensor
                Tensor of generated sequences
            label_space_size : int
                Size of label space
        Returns
        -------
            X,y : list of tuples
                Artificial ground truth dataset
                    X dim : (seq batch size)
                    y dim : (seq batch size, label space size)
        """
        
        batch_size = len(sequences)
        
        dataset = list()
        for sequence in sequences:
            # each 'token' in the sequence has a label mapping
            label_set = list()
            for token in sequence:
                # ONE HOT NOT REQUIRED; LABEL SPACE IS A SINGLE INTEGER VALUE
#                 one_hot_array = np.zeros(shape=(label_space_size))
#                 one_hot_array[random.randint(0,label_space_size-1)]=1
#                 labels = torch.IntTensor(one_hot_array)
                label_set.append(torch.randint(low=0,high=label_space_size, size=(1,), dtype=torch.long))
            dataset.append((sequence, torch.stack(label_set)))   # stack list of labels into tensors
        return dataset

In [24]:
# Test functionality
tester = Tester()
sequences, lengths = tester.build_sequences(batch_size=10, max_seq_len=10)
dataset = tester.build_sequence_tags(sequences=sequences, label_space_size=4)

Shapes - seq torch.Size([10, 10]) - lengths torch.Size([10])


In [25]:
for X, y in dataset:
    print(f'X {X.shape} ({X.dtype}) - {X}\n')
    print(f'y {y.shape} ({y.dtype}) - {y}')
    break

X torch.Size([10]) (torch.int64) - tensor([58, 78, 50,  0,  0,  0,  0,  0,  0,  0])

y torch.Size([10, 1]) (torch.int64) - tensor([[3],
        [0],
        [0],
        [3],
        [1],
        [2],
        [1],
        [2],
        [1],
        [1]])


### Utility Functions

- [ ] Build data preprocessor
- [ ] Build data loaders

In [26]:
# configuration for model building, training, evaluation process. This will be converted into yaml.
config = {'': ''}

### Models
<i>Model architectures</i><br>
<b>SVAE</b> - RNN<br>
<b>Discriminator</b> - FC NN<br>
<b>Task Learner</b> - RNN<br>
- [ ] SVAE
- [ ] Discriminator
- [ ] Task Learner

#### SVAE
To do:
 - [ ] 

In [66]:
class SVAE(nn.Module):
    """ Sentence Variational Autoencoder (Bowman et al. 2016)"""
    
    def __init__(self, vocab_size, embedding_size):
        super(SVAE, self).__init__()
        
        # TODO: fix dodgy vocab_size issue... this will be cleared up when utils implemented properly
        
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        
        self.max_sequence_length = 40    # arg
        self.pad_idx = 0
        self.eos_idx = vocab_size + 1
        self.sos_idx = vocab_size + 2
        self.unk_idx = vocab_size + 3
        
        self.vocab_size = vocab_size + 4
        
        self.z_dim = 8
        
        self.rnn_type = 'gru'
        self.bidirectional = False
        self.num_layers = 1
        self.hidden_size = 128
        
        self.embedding = nn.Embedding(self.vocab_size, embedding_size)
        self.word_dropout_rate = 0.1
        self.embedding_dropout = nn.Dropout(p=0.5)
        
        # set rnn type
        if self.rnn_type == 'gru':
            rnn = nn.GRU
        else:
            raise ValueError()
        
        # init encoder-decoder RNNs (models are identical)
        self.encoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)

        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        # Initialisation of FC layers
        # These go from encoder to latent (z) space
        self.hidden2mean = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.hidden2logv = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.z2hidden = nn.Linear(self.z_dim, self.hidden_size * self.hidden_factor)
        self.outputs2vocab = nn.Linear(self.hidden_size * (2 if self.bidirectional else 1), self.vocab_size)
    
    
    def forward(self, input_sequence, length):
        """ Forward pass through VAE """
        
        batch_size = input_sequence.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)   # trick for packed padding
        input_sequence = input_sequence[sorted_idx]
        
        # ENCODER
        input_embedding = self.embedding(input_sequence)
#         print(input_embedding.shape)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        _, hidden = self._encode(packed_input)
        
        if self.bidirectional or 1 < self.num_layers:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size * self.hidden_factor)
        else:
            # .squeeze() -> Returns a tensor with all the dimensions of input of size 1 removed.
            print(f'hidden shape before squeeze {hidden.shape}')
#             hidden = hidden.squeeze()   # doesn't work? gives wrong dimension down stream...
            pass
            print(f'hidden shape after squeeze {hidden.shape}')

        
        # Reparameterisation trick!
        z, mean, logv, std = self.reparameterise(hidden, batch_size)
        
        # DECODER
        if 0 < self.word_dropout_rate:
            prob = torch.rand(input_sequence.size())

            if torch.cuda.is_available():
                prob = prob.cuda()

            prob[(input_sequence.data - self.sos_idx) * (input_sequence.data - self.pad_idx) == 0] = 1

            decoder_input_sequence = input_sequence.clone()
            
#             print(vocab_size)
#             print(self.unk_idx)
            decoder_input_sequence[prob < self.word_dropout_rate] = self.unk_idx

#             print(decoder_input_sequence)
            input_embedding = self.embedding(decoder_input_sequence)

        input_embedding = self.embedding_dropout(input_embedding)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        
        outputs, _ = self._decode(packed_input, hidden)
        
        # process outputs
        # Process outputs
        # Unpack padded sequence
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _, reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b, s, _ = padded_outputs.size()

        # Project outputs to vocab
        # e.g. project hidden state into label space...
        logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.size(2))), dim=-1)
#         print(f'logp before view {logp.shape}\n')
#         print(f'b {b} s {s} no emb {self.embedding.num_embeddings}')
        logp = logp.view(b, s, self.embedding.num_embeddings)

        # logp - log posterior over label space; mean - tensor Gaussian mean, logv - tensor Gaussian variance, z - VAE latent space 
        return logp, mean, logv, z
    
    def to_var(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        return x
    
    def reparameterise(self, hidden, batch_size):
        """ Implement reparameterisation trick (Kingma and Welling 2014) """
        
        mean = self.hidden2mean(hidden)
        logv = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logv) 
        
        z = self.to_var(torch.randn([batch_size, self.z_dim]))
        return z * std + mean, mean, logv, std
    
    def _encode(self, x):
        """ x - pack padded sequence """
        return self.encoder_rnn(x)
    
    def _decode(self, x, hidden):
        """ x - pack padded sequence
            hidden - latent tensor"""
        return self.decoder_rnn(x, hidden)

In [67]:
# Testing functionality
vocab_size = 100
hidden_size = 128
SVAE = SVAE(vocab_size, hidden_size)

In [68]:
# Test forward pass of SVAE
svae_ff_seqs, svae_ff_lengths = Tester().build_sequences(batch_size=10, max_seq_len=40)

# Pass sequences and lengths into SVAE forward method
print(SVAE.forward(svae_ff_seqs, svae_ff_lengths))

Shapes - seq torch.Size([10, 40]) - lengths torch.Size([10])
hidden shape before squeeze torch.Size([1, 10, 128])
hidden shape after squeeze torch.Size([1, 10, 128])
(tensor([[[-4.7531, -4.8106, -4.9538,  ..., -4.6081, -4.5295, -4.5324],
         [-4.7728, -4.7408, -4.6469,  ..., -4.6056, -4.4521, -4.6779],
         [-4.6151, -4.6546, -4.6274,  ..., -4.2124, -4.4680, -4.6956],
         ...,
         [-4.6454, -4.6193, -4.7378,  ..., -4.6265, -4.5784, -4.5948],
         [-4.6454, -4.6193, -4.7378,  ..., -4.6265, -4.5784, -4.5948],
         [-4.6454, -4.6193, -4.7378,  ..., -4.6265, -4.5784, -4.5948]],

        [[-4.4980, -4.3337, -4.8377,  ..., -4.6214, -4.5822, -4.5876],
         [-4.5180, -4.0787, -4.6752,  ..., -4.5264, -4.4273, -4.4877],
         [-4.4887, -4.2805, -4.7759,  ..., -4.2257, -4.3880, -4.4419],
         ...,
         [-4.6454, -4.6193, -4.7378,  ..., -4.6265, -4.5784, -4.5948],
         [-4.6454, -4.6193, -4.7378,  ..., -4.6265, -4.5784, -4.5948],
         [-4.6454, -4.

In [30]:
# Reviewing components
print(f'Encoder: {SVAE.encoder_rnn}')
print(f'Decoder: {SVAE.decoder_rnn}')
print(f'hidden2mean: {SVAE.hidden2mean}')
print(f'hidden2logv: {SVAE.hidden2logv}')
print(f'z2hidden: {SVAE.z2hidden}')
print(f'outputs2vocab: {SVAE.outputs2vocab}')

Encoder: GRU(128, 128, batch_first=True)
Decoder: GRU(128, 128, batch_first=True)
hidden2mean: Linear(in_features=128, out_features=8, bias=True)
hidden2logv: Linear(in_features=128, out_features=8, bias=True)
z2hidden: Linear(in_features=8, out_features=128, bias=True)
outputs2vocab: Linear(in_features=128, out_features=104, bias=True)


#### Task Learner
Using PyTorch tutorial implementation (https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) for rapid development (will implement SoTA in the future)<br><br>
To do:
 - [ ] 

In [31]:
class TaskLearner(nn.Module):
    """ Task learner for NER """
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(TaskLearner, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [32]:
# Testing functionality
tasklearner = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=104, tagset_size=4)

In [33]:
# Testing forward functionality
test_seq = torch.randint(0,100,size=(10,))
print(f'Test Sequence shape: {test_seq.shape}')
print(f'Forward output:\n{tasklearner.forward(test_seq)}')

Test Sequence shape: torch.Size([10])
Forward output:
tensor([[-1.4794, -1.2650, -1.3656, -1.4492],
        [-1.4865, -1.2843, -1.3209, -1.4692],
        [-1.4257, -1.3145, -1.3280, -1.4870],
        [-1.3941, -1.3571, -1.3332, -1.4658],
        [-1.3769, -1.3420, -1.3499, -1.4826],
        [-1.4530, -1.2435, -1.4825, -1.3834],
        [-1.5051, -1.2764, -1.4306, -1.3479],
        [-1.4349, -1.3009, -1.3679, -1.4485],
        [-1.5529, -1.2953, -1.2882, -1.4322],
        [-1.5578, -1.2940, -1.3844, -1.3290]], grad_fn=<LogSoftmaxBackward>)


In [34]:
# Testing basic training cycle
seqs, lens = tester.build_sequences(batch_size=10, max_seq_len=10)   # Tester initialised at start of notebook
dataset = tester.build_sequence_tags(sequences=seqs, label_space_size=4)

Shapes - seq torch.Size([10, 10]) - lengths torch.Size([10])


In [35]:
# Generate vocab off of generated sequences
vocab = list()
for seq in seqs:
      vocab.extend(seq.tolist())
vocab = list(set(vocab))
print(vocab)

[0, 4, 5, 6, 7, 8, 15, 16, 18, 20, 21, 22, 24, 25, 27, 29, 30, 31, 37, 39, 41, 42, 44, 46, 50, 51, 52, 53, 54, 61, 65, 69, 73, 74, 75, 80, 81, 87, 88, 89, 91, 92, 96]


In [36]:
# Test training routine
# vocab_size is max int in vocab ints + 1 as 0 is included...
model = TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=max(vocab)+1, tagset_size=4)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [37]:
with torch.no_grad():
    inputs = seqs[0]
    # forward pass to get output scores
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.4035, -1.4262, -1.4404, -1.2830],
        [-1.3603, -1.4606, -1.5082, -1.2378],
        [-1.3004, -1.5160, -1.5087, -1.2490],
        [-1.2736, -1.5231, -1.5250, -1.2570],
        [-1.2643, -1.5132, -1.5407, -1.2618],
        [-1.2610, -1.5026, -1.5530, -1.2642],
        [-1.2600, -1.4942, -1.5617, -1.2654],
        [-1.2601, -1.4881, -1.5675, -1.2658],
        [-1.2606, -1.4838, -1.5714, -1.2658],
        [-1.2612, -1.4809, -1.5741, -1.2657]])


In [38]:
for epoch in range(5):
    print(f'Epoch: {epoch}')
    for seq, tags in dataset:
#         print(seq,tags)

        model.zero_grad()
        seq_in = seq
        targets = tags.view(-1)
#         print(targets.shape)
#         print(targets.view(-1).shape)
        
        tag_scores = model(seq_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
        print(loss.data)

Epoch: 0
tensor(1.3794)
tensor(1.4134)
tensor(1.4450)
tensor(1.4319)
tensor(1.3841)
tensor(1.4103)
tensor(1.4326)
tensor(1.3774)
tensor(1.4121)
tensor(1.3973)
Epoch: 1
tensor(1.4099)
tensor(1.3550)
tensor(1.4328)
tensor(1.4307)
tensor(1.3450)
tensor(1.3413)
tensor(1.3767)
tensor(1.3472)
tensor(1.4051)
tensor(1.3582)
Epoch: 2
tensor(1.4198)
tensor(1.3316)
tensor(1.4144)
tensor(1.4278)
tensor(1.3143)
tensor(1.3202)
tensor(1.3321)
tensor(1.3245)
tensor(1.3933)
tensor(1.3250)
Epoch: 3
tensor(1.4179)
tensor(1.3163)
tensor(1.3944)
tensor(1.4218)
tensor(1.2864)
tensor(1.3108)
tensor(1.2927)
tensor(1.3037)
tensor(1.3802)
tensor(1.2939)
Epoch: 4
tensor(1.4133)
tensor(1.3031)
tensor(1.3742)
tensor(1.4149)
tensor(1.2595)
tensor(1.3048)
tensor(1.2559)
tensor(1.2830)
tensor(1.3665)
tensor(1.2634)


In [ ]:
with torch.no_grad():
    inputs, _ = dataset[0]
    tag_scores = model(inputs)

    print(tag_scores)

#### Discriminator
To do:
 - [ ] 

In [39]:
class Discriminator(nn.Module):
    """ Adversary architecture for discriminator module 
    
    Tensor needs to be FloatTensor; sequences and tag sets are LongTensor
    """
    
    def __init__(self, z_dim=8):
        super(Discriminator, self).__init__()
        
        self.z_dim = z_dim    # latent space dimension (will be the same shape as the encoder output from VAE)
        
        self.net = nn.Sequential(
                                nn.Linear(z_dim, 128),
                                nn.ReLU(True),
                                nn.Linear(128, 128),
                                nn.ReLU(True),
                                nn.Linear(128,1),
                                nn.Sigmoid()
                                )
        # Exe
        self.weight_init()
        
    def weight_init(self):
        """ Weight initialisation
        
        Using Xavier uniform initialisation rather than Kaiming (I think that is more focused for CV? TODO: investigate)
        See: https://pytorch.org/cppdocs/api/function_namespacetorch_1_1nn_1_1init_1ace282f75916a862c9678343dfd4d5ffe.html
        """
        for block in self._modules:
            for m in self._modules[block]:
                if type(m) == nn.Linear:
                    torch.nn.init.xavier_uniform_(m.weight)
                    m.bias.data.fill_(0.01)
    
    def forward(self, z):
        """ Forward pass through discriminator
        
        Arguments
        --------
            z : tensor
                Tensor derived from SVAE latent space
        """
        return self.net(z.type(torch.FloatTensor))

In [45]:
# Testing functionality
z_dim = 8
discriminator = Discriminator(z_dim=z_dim)

# Pass random integer tensor through forward pass of discriminator
rand_tensor = torch.randint(low=0, high=100,size=(z_dim,))
print(f'Input: {rand_tensor}')
print(f'Output: {discriminator.forward(rand_tensor)}')

Input: tensor([59, 92,  6,  7, 99, 33, 74, 59])
Output: tensor([0.2480], grad_fn=<SigmoidBackward>)


In [47]:
# Testing training functionality for discriminator
dsc_bce_loss = nn.BCELoss()
dsc_optim = optim.Adam(discriminator.parameters(), lr=0.001)

discriminator.train()

preds = discriminator(rand_tensor)
real_labels = torch.ones(preds.size(0))
loss = dsc_bce_loss(preds, real_labels)

print(loss)

tensor(1.3945, grad_fn=<BinaryCrossEntropyBackward>)


## Sampler Routine
Active learning based sample selection for task learner

In [48]:
# Code copied from VAAL and modified for sequence data
class Sampler:
    """ Adversary sampler """
    def __init__(self, budget):
        self.budget = budget
        
    def sample(self, vae, discriminator, data, cuda):
        """ Selective sampling algorithm
        
        Arguments
        ---------
            vae : torch model
                VAE model
            discriminator : torch model
                discriminator model
            data : tensor
                Image data
            cuda : boolean
                GPU flag
        Returns
        -------
            querry_pool_indices: int, list
                List of indices corresponding to sorted (top-K) samples to be sampled from
        """
        all_preds = []
        all_indices = []

        for images, _, indices in data:
            if cuda:
                images = images.cuda()

            with torch.no_grad():
                _, _, mu, _ = vae(images)
                preds = discriminator(mu)

            preds = preds.cpu().data
            all_preds.extend(preds)
            all_indices.extend(indices)

        all_preds = torch.stack(all_preds)
        all_preds = all_preds.view(-1)
        # need to multiply by -1 to be able to use torch.topk 
        all_preds *= -1

        # select the points which the discriminator things are the most likely to be unlabeled
        _, querry_indices = torch.topk(all_preds, int(self.budget))
        querry_pool_indices = np.asarray(all_indices)[querry_indices]

        return querry_pool_indices

In [49]:
# Testing functionality
sampler = Sampler(budget=10)

### Training Routine
<i> Pseudo code</i>

```python
for epoch in max_epochs:
        train(task learner)
            get preds
            calc loss
            zero grads
            backpropigate loss
            update model parameters
        for step in max_steps:
            train(SVAE)
        for step in max_steps:
            train(discriminator)
```
To do:
 - [ ] Implement training cycle for task learn
 - [ ] Implement training cycle for VAE
 - [ ] Implement training cycle for discriminator

In [64]:
class Solver:
    def __init__(self, task_learner, vae, discriminator):
        self.epochs = 2
        self.vae_steps = 2
        self.discriminator_steps = 2
        
        # Get models
        # init else where?
        self.task_learner =  TaskLearner(embedding_dim=128, hidden_dim=128, vocab_size=max(vocab)+1, tagset_size=4)
        self.svae = SVAE(max(vocab)+1, hidden_size=128)
        self.discriminator = Discriminator(z_dim=10)   # z_dim will be the latent output of the vae encoder... currently just the seq length
        
        # Loss functions
        self.nll_loss = nn.NLLLoss()   # used in: TL
        self.bce_loss = nn.BCELoss()   # used in: Discriminator
        self.xxx_loss = 0    # used in: SVAE

        # optimiser
        self.optim_lr = 0.1
        self.tl_optim = optim.SGD(model.parameters(), lr=self.optim_lr)   # todo: revisit learning rate
        self.svae_optim = None
        self.disc_optim = optim.Adam(self.discriminator.parameters(), lr=0.01)   # todo: revisit learning rate
        
        # Will need to implement this in the future, need to build generator function rather than rely on Torch implementationss?
        self.data_loader = 'generator'
        self.seqs, self.lens = tester.build_sequences(batch_size=10, max_seq_len=10)   # Tester initialised at start of notebook
        self.dataset = tester.build_sequence_tags(sequences=seqs, label_space_size=4)
        
        # Will need to implement this in the future, inherit the Sampler class?
        self.sampler = 'sampler'
        
    def train(self):
        """ Performs model training """
        
        # turn on .train() mode
        self.svae.train()
        self.task_learner.train()
        self.discriminator.train()
        
        for epoch in range(self.epochs):
            print(f'\nEpoch: {epoch}')
            
            # these will be batches in the future, not single sequence/tag sets...
            # build generator to wrap around sequence/tag sets and then just call next() on them at each epoch to get a batch...
            # need to build that in the future to partition based on the unlabelled/labelled splits for AL experiments
            for seq, tags in self.dataset:
#                 print(seq, tags)
            
                # Train Task Learner
                self.tl_train_step(seq, tags)
            
            
                # Train VAE
                for step in range(self.svae_steps):
                    print(f'SVAE Step: {step}')
                    self.svae_train_step()

                # Train Discriminator
                for step in range(self.discriminator_steps):
                    print(f'Discriminator Step: {step}')

                    self.disc_train_step(seq)
            clear_output(wait=True)
                
    def tl_train_step(self, seqs, tags):
        """ Perform training step of task learner 
        
        Tensor dtype is Long for tl
        """
#         print(seq.dtype)
#         assert seq.dtype == 'torch.int64'
#         assert tags.dtype == 'torch.int64'
        
        # zero grad as they accumulate in PyTorch
        self.task_learner.zero_grad()
        preds = self.task_learner(seqs)
        tl_loss = self.nll_loss(preds, tags.view(-1).type(torch.LongTensor))   # flatten tags as they are a 1D tensor rather than 2D
        tl_loss.backward()
        self.tl_optim.step()
        
        print(f'Task learning loss: {loss.data:0.4f}')
    
    def svae_train_step(self, seqs, lengths, tags):
        """ Perform training step of sentence variational autoencoder """
#         print(self.svae)
        
        # Forward pass
        logp, mean, logv, z = self.svae(seq, lengths)
        
        # Loss calculation
        NLL_loss, KL_loss, KL_weight = loss_fn(logp, tags, lengths, mean, logv, anneal_function, step, k, x0)
        
        
        
        
    
    def disc_train_step(self, seqs):
        """ Perform training step of adversarial discriminatory 
        
        Note: The loss function will be an aggregate across labelled and unlabelled samples (currently just labelled)
        
        Tensor dtype is FloatTensor for discriminator
        """
        
#         assert seq.dtype == 'torch.float64'
        
#         print(seq)
#         print(seq.dtype)
        
        preds = self.discriminator(seqs)
#         print(preds)
        real_labels = torch.ones(preds.size(0))
        disc_loss = self.bce_loss(preds, real_labels)
        disc_loss.backward()
        self.disc_optim.step()
        
        print(f'Discriminator loss: {loss.data}')
    

In [65]:
# Test training 
slvr = Solver(task_learner='tl', vae='vae', discriminator='disc')
slvr.train()


Epoch: 1
Task learning loss: 1.3945
VAE Step: 0
vae
VAE Step: 1
vae
Discriminator Step: 0
Discriminator loss: 1.3944728374481201
Discriminator Step: 1
Discriminator loss: 1.3944728374481201
Task learning loss: 1.3945
VAE Step: 0
vae
VAE Step: 1
vae
Discriminator Step: 0
Discriminator loss: 1.3944728374481201
Discriminator Step: 1
Discriminator loss: 1.3944728374481201
Task learning loss: 1.3945
VAE Step: 0
vae
VAE Step: 1
vae
Discriminator Step: 0
Discriminator loss: 1.3944728374481201
Discriminator Step: 1
Discriminator loss: 1.3944728374481201
Task learning loss: 1.3945
VAE Step: 0
vae
VAE Step: 1
vae
Discriminator Step: 0
Discriminator loss: 1.3944728374481201
Discriminator Step: 1
Discriminator loss: 1.3944728374481201
Task learning loss: 1.3945
VAE Step: 0
vae
VAE Step: 1
vae
Discriminator Step: 0
Discriminator loss: 1.3944728374481201
Discriminator Step: 1
Discriminator loss: 1.3944728374481201
Task learning loss: 1.3945
VAE Step: 0
vae
VAE Step: 1
vae
Discriminator Step: 0
Disc

### Inference Routine
To do:
 - [ ] 


### Sampling Routine
To do:
 - [ ] 